In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import data_processing as dp
from scipy import signal
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt
import pywt
import os
import time
import h5py
from itertools import combinations

In [ ]:
for dirname, _, filenames in os.walk('.\data'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

In [ ]:
path2 = '.\data\正常\G11_Walking_trial_4_emg.csv'

In [17]:
Data = pd.read_csv('./processed data/dataframe_W500_S125_DWTLmax_samelabel_sc.csv')

# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
drop = 'G07_Freezing_Trial1_trial_1_emg.csv'
drop2= 'P812_M050_2_B_FoG_trial_2_emg.csv'
drop3= 'P812_M050_2_B_FoG_trial_1_emg.csv'
ind_drop = (df.columns!=drop)# & (df.columns!=drop2) & (df.columns!=drop3)
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)

files = np.concatenate([np.array(df.columns),np.array(df3.loc[:,0])])
ind = Data.File.isin(files)
ind2 = Data.File == drop
#ind = (Data.File != drop) & (Data.File != drop2)
Data_sel = Data[ind]
Data_rest = Data[ind2]
#ind2 = Data_rest.File == drop
#Data_rest = Data_rest[ind2]
#Data_rest

In [181]:
feature_col = ['_IEMG','_MAV','_SSI','_VAR','_RMS',
               '_WL','_ZC','_SSC','_WAMP','_skew','_Acti','_AR','_HIST','_MDF']

feature_all = Data_sel.iloc[:,1:-1]
#ind_temp1 = feature_all.columns.str.contains('_mDWT')
#ind_temp2 = feature_all.columns.str.contains('_Coe')
#ind_temp3 = feature_all.columns.str.contains('_Scale')
#ind_temp4 = feature_all.columns.str.contains('_HIST')
#ind_temp = ind_temp1|ind_temp2|ind_temp3|ind_temp4
ind_temp = feature_all.columns.str.contains('_mDWT')
feature = feature_all.loc[:,~ind_temp]
#feature = Data_sel.iloc[:,1:-1]
#feature = Data_sel.iloc[:,:-2]
y = Data_sel.Label
feature2_all = Data_rest.iloc[:,1:-1]
#feature2 = feature2_all.loc[:,~ind_temp]
feature2 = Data_rest.iloc[:,1:-1]
#feature2 = Data_rest.iloc[:,:-2]
y2 = Data_rest.Label

In [18]:
feature = pd.DataFrame()
feature2 = pd.DataFrame()
y = pd.DataFrame()
y2 = pd.DataFrame()
m = 0
for i in set(Data_sel.File):
    ind = Data_sel.File==i
    temp = Data_sel[ind]
    for j in set(temp.Label):
        ind2 = temp.Label == j
        temp2 = temp[ind2]
        l = len(temp2)
        feature = pd.concat([feature,temp2.iloc[:int(0.8*l),1:-1]])
        y = pd.concat([y,temp2.iloc[:int(0.8*l),0]])
        feature2 = pd.concat([feature2,temp2.iloc[int(0.8*l):,1:-1]])
        y2 = pd.concat([y2,temp2.iloc[int(0.8*l):,0]])
y = np.array(y)[:,0]
y2 = np.array(y2)[:,0]
#ind_temp = feature.columns.str.contains('_mDWT')
#feature = feature.loc[:,~ind_temp]
#feature2 = feature2.loc[:,~ind_temp]

In [184]:
feature

,LEFT_TA_IEMG,LEFT_TS_IEMG,LEFT_BF_IEMG,LEFT_RF_IEMG,RIGHT_TA_IEMG,RIGHT_TS_IEMG,RIGHT_BF_IEMG,RIGHT_RF_IEMG,LEFT_TA_SSI,LEFT_TS_SSI,...,RIGHT_BF_MDF,RIGHT_RF_MDF,LEFT_TA_MNF,LEFT_TS_MNF,LEFT_BF_MNF,LEFT_RF_MNF,RIGHT_TA_MNF,RIGHT_TS_MNF,RIGHT_BF_MNF,RIGHT_RF_MNF
13519,88.962654,77.34741,57.224567,62.383404,91.892540,47.207516,46.380474,68.208664,52.591140,36.242460,...,203.12500,70.31250,140.045230,181.662384,150.619937,137.972132,125.860405,236.136973,211.541490,136.647239
13520,86.452050,74.36432,60.487090,61.635277,94.097824,45.952827,42.839050,65.562744,51.380930,34.295430,...,171.87500,70.31250,138.435458,178.426941,110.770259,127.892318,117.876622,242.906437,203.716326,132.833129
13521,86.365220,71.03250,67.075970,58.983920,90.001495,44.022910,42.633220,65.724830,51.487750,32.098423,...,171.87500,66.40625,139.166098,168.600511,101.430259,120.203195,111.976777,237.209126,203.901220,127.235219
13522,95.490320,67.59361,67.169230,57.006170,85.765870,42.373833,39.899174,61.722763,60.989697,30.739939,...,160.15625,62.50000,143.463454,165.548731,103.543113,121.120256,117.630899,238.297868,195.087730,121.792518
13523,105.050766,64.81915,65.485150,55.373486,82.777890,41.133270,38.543266,57.451298,71.195755,29.257748,...,160.15625,66.40625,136.519421,163.331144,105.638626,121.109534,110.014394,228.041846,195.617437,130.177478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43250,64.833595,96.78418,48.135414,63.579323,59.692368,63.578720,69.708855,54.296165,30.981726,57.836420,...,74.21875,11.71875,120.817626,155.749876,130.621929,66.624950,117.651555,138.848233,94.909326,38.530130
43251,63.496452,94.05290,38.649220,70.976906,58.460938,64.411950,74.785120,56.792942,30.353344,56.520947,...,70.31250,11.71875,119.347708,156.292783,131.199122,61.517871,103.881146,139.020772,87.624188,40.055292
43252,61.259148,87.55627,35.732845,76.422750,54.117054,68.629220,79.662810,56.222706,28.747637,52.033546,...,70.31250,11.71875,114.577111,155.016133,127.824618,66.618924,123.716738,138.830209,86.537925,39.088569
43253,57.529190,84.12217,33.024437,80.876680,48.072784,71.334130,84.322530,57.591923,25.658537,50.555576,...,74.21875,11.71875,118.317610,155.892168,128.157184,66.951979,111.311017,139.915881,91.220167,39.460164


In [ ]:
path = './processed data/data_set_after_window_withoutSC.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]

In [1]:
import imp
imp.reload(dp)

NameError: name 'dp' is not defined

In [3]:
import ctypes
player = ctypes.windll.kernel32
player.Beep(1000,200)

1

In [128]:
(Data_rest.File=='P812_M050_2_B_FoG_trial_1_emg.csv').sum()

0

In [ ]:
width = 256
threshold_WAMP = 30
threshold_ZC = 0
threshold_SSC = 1
bins=9
bound = 70
HIST_range = (-bound,bound)

In [4]:
path = './processed data/data_set_after_window_W128_S64_sameLabel_rect.hdf5'
with h5py.File(path,'r') as f:
    x = f['data'][...]
    y = f['label2'][...]
#feature = dp.generate_feature(x,threshold_WAMP=threshold_WAMP,
#                              threshold_ZC=threshold_ZC,
#                              threshold_SSC=threshold_SSC,
#                              bins=bins,ranges=HIST_range)
#feature2 = dp.generate_feature(x2)
player.Beep(1000,200)

1

In [ ]:
with h5py.File('./processed data/nfeatures_W256_S64_WAMP30.hdf5','r') as f:
    feature = f['features'][...]
    y = f['labels'][...]

In [ ]:
feature2,y2 = dp.pipeline_feature(path2,width=256,stride=64,
                                  scaler=False,
                                  threshold_WAMP=threshold_WAMP,
                                  threshold_ZC=threshold_ZC,
                                  threshold_SSC=threshold_SSC,
                                  bins=bins,ranges=HIST_range)

In [ ]:
with h5py.File('./processed data/nfeatures_rest_W256_S64.hdf5','r') as f:
    feature2 = f['features'][...]
    y2 = f['labels'][...]

In [ ]:
sc = StandardScaler(with_mean=True)
feature_sc = sc.fit_transform(feature)
feature2_sc = sc.transform(feature2)

# Model

In [4]:
import tensorflow as tf
from tensorflow.keras import layers,Model,callbacks,regularizers,models
from sklearn.model_selection import train_test_split,cross_validate
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder,normalize,MinMaxScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE,BorderlineSMOTE,ADASYN,SVMSMOTE

In [5]:
tf.executing_eagerly()

True

In [12]:
def train_model(model,feature,y,binary=True,file=None):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==6))
        #ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y[ind].copy()
        ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}
    x_full,x_test,y_full,y_test = train_test_split(np.array(feature)[ind,:],y_01,
                                                   test_size=0.2,
                                                   random_state=123,
                                                   shuffle=True)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.25,
                                                       random_state=555,
                                                       shuffle=True)
    
    #sm = BorderlineSMOTE(random_state=50,kind='borderline-2')
    #sm = SMOTE(random_state=50)
    #print(y_full.shape)
    #x_full,y_full = sm.fit_resample(x_full,y_full)
    #print(y_full_n.shape)
    sc = StandardScaler(with_mean=True)
    #sc = MinMaxScaler()
    x_train = sc.fit_transform(x_full)
    pca = PCA(n_components=100)
    #x_train = pca.fit_transform(x_train)
    #x_valid = sc.transform(x_valid)
    x_test = sc.transform(x_test)
    #x_test = pca.transform(x_test)
    #x_train = x_full
    
    early_stopping = callbacks.EarlyStopping(patience = 20,
                                             monitor = 'val_loss', 
                                             restore_best_weights=True)
    history = model.fit(x_train,y_full,validation_data=[x_test,y_test],
                        epochs=100,batch_size=32,class_weight=cw,
                        #callbacks=[early_stopping],
                        shuffle=True)
    
    if binary:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #test = metrics.accuracy_score(y_test,y_pred_t>0.5)
        
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(y_valid,np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))
        #train = metrics.accuracy_score(y_full,y_pred_ta>0.5)
        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
        
        #print('train: \n',metrics.confusion_matrix(y_full,y_pred_ta>0.5))
        #print('test: \n',metrics.confusion_matrix(y_test,y_pred_t>0.5))

    else:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
    print('test:%f'%test)
    #print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,test,sc,pca

def test_model(model,feature,y,sc,pca,binary=True):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==3)|(y==4)|(y==6))
        #ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y[ind].copy()
        ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}

    #print(y_01)
    feature=sc.transform(feature[ind])
    #feature = feature[ind]
    #feature=pca.transform(feature)
    y_pred=model.predict(feature)
    test = metrics.accuracy_score(np.argmax(y_01,axis=1),np.argmax(y_pred,axis=1))
    #test = metrics.accuracy_score(y_01,y_pred>0.5)
    
    print('acc:%f'%test)
    print(metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(y_pred,axis=1)))
    #print(metrics.confusion_matrix(y_01,y_pred>0.5))
    return test

def sparse_cost_sensitive_loss (y_true,y_pred):
    #cost_matrix = tf.constant([[0,1.,1,1.],
    #              [2,0,5,5],
    #              [1,1,0,1],
    #              [1.,2.,1,0]])
    cost_matrix = tf.constant([[0,2.,2],
                  [1,0,1],
                  [1.0,1.,0]])
    #cost_matrix = tf.constant([[0,1.],
    #              [5.,0]])
    batch_cost_matrix = tf.nn.embedding_lookup(cost_matrix, tf.argmax(y_true,axis=1))
    eps = 1e-6
    probability = tf.clip_by_value(y_pred, eps, 1-eps)
    cost_values = tf.math.log(1-probability)*batch_cost_matrix
    loss = tf.reduce_mean(-tf.reduce_sum(cost_values, axis=1))
    return loss

## ANN

In [6]:
ind = ((y==1)|(y==2)|(y==3)|(y==6))
#ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
ind_f = [0,1,6,42,46,57,62]
#y_01 = y[ind].copy()
#y_01[y_01==1]=0
#y_01[y_01==2]=1
#y_01[y_01==3]=2
#y_01[y_01==6]=3
y_01 = y[ind].copy()
#y_01[ind] = 1
oh_ec = OneHotEncoder()
y_oh = oh_ec.fit_transform(y_01[:,np.newaxis]).toarray()
x_full,x_test,y_full,y_test = train_test_split(feature.loc[ind,:],y_01,test_size=0.2,random_state=123,shuffle=False)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555,shuffle=True)

In [7]:
input_ = layers.Input(shape=feature.shape[1:])#feature.shape[1:]
l1 = layers.Dense(128,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(input_)
#drop1 = layers.Dropout(0.2)(l1)
bn1 = layers.BatchNormalization()(l1)
l2 = layers.Dense(64,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(bn1)
#drop2 = layers.Dropout(0.2)(l2)
bn2 = layers.BatchNormalization()(l2)
l3 = layers.Dense(32,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(bn2)
#drop3 = layers.Dropout(0.2)(l3)
bn3 = layers.BatchNormalization()(l3)
l4 = layers.Dense(16,activation='elu',
                 #kernel_regularizer = regularizers.l2(0.001),
                 )(bn3)
#drop4 = layers.Dropout(0.2)(l4)
#l5 = layers.Dense(8,activation='relu')(drop4)
#drop5 = layers.Dropout(0.5)(l5)
output = layers.Dense(3,activation='softmax')(l4)
model = Model(inputs=[input_],outputs=[output])

In [19]:
model = models.Sequential()
#model.add(layers.BatchNormalization())
model.add(layers.Dense(128,#activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                       #use_bias=False
                 ))
model.add(layers.Dropout(0.2))
#model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))
model.add(layers.Dense(64,#activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                      # use_bias=False
                 ))
model.add(layers.Dropout(0.2))
#model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))
model.add(layers.Dense(32,#activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                       #use_bias=False
                 ))
model.add(layers.Dropout(0.2))
#model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))
model.add(layers.Dense(16,#activation='elu',
                 #kernel_regularizer = regularizers.l2(0.001),
                       #use_bias=False
                 ))
model.add(layers.Dropout(0.2))
#model.add(layers.BatchNormalization())
model.add(layers.Dense(3,activation='softmax'))

In [20]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train model

In [21]:
#sc = StandardScaler(with_mean=True)
#feature_sc = sc.fit_transform(feature)
#feature2_sc = sc.transform(feature2)
#pca = PCA(n_components=160,copy=True)
#feature_pca = pca.fit_transform(feature)
#feature2_pca = pca.transform(feature2)
train,test,sc,pca = train_model(model,feature,np.array(y),False)
#rest = test_model(model,feature2,np.array(y2),sc)
#acc = [train,valid,test,rest]

Train on 3476 samples, validate on 870 samples
Epoch 1/100
3476/3476 [==============================] - 1s 322us/sample - loss: 0.8823 - accuracy: 0.6326 - val_loss: 0.5863 - val_accuracy: 0.7471
Epoch 2/100
3476/3476 [==============================] - 0s 98us/sample - loss: 0.6698 - accuracy: 0.7149 - val_loss: 0.5494 - val_accuracy: 0.7644
Epoch 3/100
3476/3476 [==============================] - 0s 106us/sample - loss: 0.6204 - accuracy: 0.7394 - val_loss: 0.5271 - val_accuracy: 0.7632
Epoch 4/100
3476/3476 [==============================] - 0s 106us/sample - loss: 0.5882 - accuracy: 0.7546 - val_loss: 0.5018 - val_accuracy: 0.7874
Epoch 5/100
3476/3476 [==============================] - 0s 103us/sample - loss: 0.5541 - accuracy: 0.7762 - val_loss: 0.5198 - val_accuracy: 0.7759
Epoch 6/100
3476/3476 [==============================] - 0s 101us/sample - loss: 0.5145 - accuracy: 0.7886 - val_loss: 0.5212 - val_accuracy: 0.7851
Epoch 7/100
3476/3476 [==============================] - 0s 

3476/3476 [==============================] - 0s 142us/sample - loss: 0.1112 - accuracy: 0.9568 - val_loss: 0.8716 - val_accuracy: 0.8115
Epoch 56/100
3476/3476 [==============================] - 0s 95us/sample - loss: 0.1251 - accuracy: 0.9548 - val_loss: 0.8833 - val_accuracy: 0.8126
Epoch 57/100
3476/3476 [==============================] - 0s 96us/sample - loss: 0.1228 - accuracy: 0.9534 - val_loss: 0.8004 - val_accuracy: 0.8218
Epoch 58/100
3476/3476 [==============================] - 0s 103us/sample - loss: 0.1122 - accuracy: 0.9583 - val_loss: 0.8635 - val_accuracy: 0.8195
Epoch 59/100
3476/3476 [==============================] - 0s 99us/sample - loss: 0.1187 - accuracy: 0.9574 - val_loss: 0.8567 - val_accuracy: 0.8138
Epoch 60/100
3476/3476 [==============================] - 0s 100us/sample - loss: 0.1076 - accuracy: 0.9594 - val_loss: 0.8144 - val_accuracy: 0.8218
Epoch 61/100
3476/3476 [==============================] - 0s 115us/sample - loss: 0.1049 - accuracy: 0.9606 - val_lo

In [22]:
test_model(model,np.array(feature2),np.array(y2),sc,pca,False)

acc:0.729167
[[ 96  87  65]
 [ 51 470  34]
 [ 31  31 239]]


0.7291666666666666

In [38]:
model.save('./model/ann.h5')

In [43]:
ind = np.array((y==0)|(y==1)|(y==2)|(y==6))
#feature
ind

array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [ ]:
#test_model(model,feature2_sc,np.array(y2))
ind = ((y==1)|(y==2)|(y==6))
y_01 = y[ind].copy()
oc = OneHotEncoder()
y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
pred=model.predict(feature_sc[ind,:])
np.argmax(pred,axis=1)
np.argmax(y_01,axis=1)
metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(pred,axis=1))

In [ ]:
#acc_ann = pd.DataFrame(acc,columns=['dnn1'],index=['train','valid','test','rest data'])
#acc_ann_com=pd.concat([acc_ann_com,acc_ann.T])
#acc_ann.loc['drop_mDWT',:]=[train,valid,test,rest]
acc_ann_com.loc['dnn2',:]=acc
acc_ann_com

# test feature from Left or Right

In [ ]:
input_ = layers.Input(shape=feature.loc[:,ind_temp].shape[1:])
l1 = layers.Dense(128,activation='elu')(input_)
drop1 = layers.Dropout(0.2)(l1)
l2 = layers.Dense(64,activation='elu')(drop1)
drop2 = layers.Dropout(0.2)(l2)
#l3 = layers.Dense(32,activation='elu')(drop2)
#drop3 = layers.Dropout(0.2)(l3)
#l4 = layers.Dense(16,activation='selu')(l3)
#drop4 = layers.Dropout(0.2)(l4)
output = layers.Dense(1,activation='sigmoid')(drop2)
model = Model(inputs=[input_],outputs=[output])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [209]:
acc={}
cols = ['LEFT','RIGHT']

#sc = StandardScaler(with_mean=True)

for col in cols:
    ind_temp=feature.columns.str.contains(col)
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    #acc[col] = [train,valid,test,acc_rest]
    print(acc_rest)

Train on 23125 samples, validate on 5782 samples
Epoch 1/200
23125/23125 [==============================] - 4s 167us/sample - loss: 1.9657 - accuracy: 0.5308 - val_loss: 1.3537 - val_accuracy: 0.6404
Epoch 2/200
23125/23125 [==============================] - 3s 109us/sample - loss: 1.3913 - accuracy: 0.6479 - val_loss: 1.1555 - val_accuracy: 0.7008
Epoch 3/200
23125/23125 [==============================] - 3s 110us/sample - loss: 1.2045 - accuracy: 0.7052 - val_loss: 1.0439 - val_accuracy: 0.7771
Epoch 4/200
23125/23125 [==============================] - 2s 105us/sample - loss: 1.1076 - accuracy: 0.7394 - val_loss: 0.9467 - val_accuracy: 0.7608
Epoch 5/200
23125/23125 [==============================] - 2s 86us/sample - loss: 1.0526 - accuracy: 0.7501 - val_loss: 0.9212 - val_accuracy: 0.7508
Epoch 6/200
23125/23125 [==============================] - 2s 93us/sample - loss: 0.9990 - accuracy: 0.7677 - val_loss: 0.8937 - val_accuracy: 0.7975
Epoch 7/200
23125/23125 [======================

23125/23125 [==============================] - 2s 71us/sample - loss: 0.8517 - accuracy: 0.8045 - val_loss: 0.8904 - val_accuracy: 0.8210
Epoch 9/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.8091 - accuracy: 0.8131 - val_loss: 1.0212 - val_accuracy: 0.8426
Epoch 10/200
23125/23125 [==============================] - 2s 72us/sample - loss: 0.7750 - accuracy: 0.8260 - val_loss: 1.0699 - val_accuracy: 0.8438
Epoch 11/200
23125/23125 [==============================] - 2s 72us/sample - loss: 0.7591 - accuracy: 0.8365 - val_loss: 0.9963 - val_accuracy: 0.8513
Epoch 12/200
23125/23125 [==============================] - 2s 70us/sample - loss: 0.7071 - accuracy: 0.8416 - val_loss: 1.0759 - val_accuracy: 0.8464
Epoch 13/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.6955 - accuracy: 0.8444 - val_loss: 1.0100 - val_accuracy: 0.8442
Epoch 14/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.7017 - accuracy: 0.8478

In [ ]:
acc_lr=pd.DataFrame(acc,index=['train','valid','test','rest data'])#.to_csv('./results/acc_lr_ann.csv')
#acc_com=pd.concat([acc_ann_com.drop(['dnn','dnn1'],'index'),acc_lr.T])
acc_com

# test features from 2 of 8 signals

In [ ]:
#test features from 2 of 8 signals

acc_2f={}
cols = [ 'LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF',
        'RIGHT_TA','RIGHT_TS','RIGHT_BF', 'RIGHT_RF']


for p in combinations(cols[:4],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)
    
for p in combinations(cols[4:],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)

In [ ]:
#acc_2f_ann = pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T
#acc_com = pd.concat([acc_com,acc_2f_ann])
acc_com.to_csv('./results/dropna/acc_com_ann.csv')

In [ ]:
pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T

# Test on some of rest

In [ ]:
some_of_rest = ['正常/P940_MSham_B_Walking_trial_6_emg.csv',
                '正常/P940_M050_B_Walking_trial_4_emg.csv',
                '正常/P812_M100_A_Walking_trial_3_emg.csv',
                '正常/P645_M050_A_Walking_trial_3_emg.csv',
                '正常/P623_Msham_B_Walking_trial_2_emg.csv',
                '正常/P551_M50_B_Walking_trial_6_emg.csv',
                'P379_M050_2_OFF_A_FoG_trial_1_emg.csv',
                'P551_M050_2_B_FoG_trial_2_emg.csv']
#booster = xgb.Booster()
#booster.load_model('./model/XGBoost_W256_S64_Left.json')
#model = xgb.XGBClassifier()
#model._Booster = booster
acc = []
columns=['LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF']
for file in some_of_rest:
    path = './data/'+file
    feature2,y2 = dp.pipeline_feature(path,width=256,stride=64,scaler=False,
                                      threshold_WAMP=threshold_WAMP,
                                      threshold_ZC=threshold_ZC,
                                      threshold_SSC=threshold_SSC,
                                      bins=bins,
                                      ranges=HIST_range,
                                      show_para=False,
                                      filt = 250)
    feature2_sc = sc.transform(feature2)
    acc += [test_model(model,feature2_sc,y2)]
print(acc)

In [ ]:
y_pred=model.predict(feature2_sc)
metrics.accuracy_score(y2,y_pred>0.5)

## RNN

In [92]:
files = np.array(df.columns)
X = []
Y = []
#sc = StandardScaler(with_mean=False)
i = 0
start = time.time()
for file in files:
    emg_data = pd.read_csv('./data/'+file)
    #emg_data = emg_data.fillna({'LEFT_TA':emg_data.LEFT_TA.mean(),
    #                            'LEFT_TS':emg_data.LEFT_TS.mean(),
    #                            'LEFT_BF':emg_data.LEFT_BF.mean(),
    #                            'LEFT_RF':emg_data.LEFT_RF.mean(),
    #                            'RIGHT_TA':emg_data.RIGHT_TA.mean(),
    #                            'RIGHT_TS':emg_data.RIGHT_TS.mean(),
    #                            'RIGHT_BF':emg_data.RIGHT_BF.mean(),
    #                            'RIGHT_RF':emg_data.RIGHT_RF.mean()})
    #emg_data = emg_data[emg_data.Label1 == emg_data.Label2].reset_index(drop=True)
    emg_data = emg_data.dropna().reset_index(drop=True)
    x = np.array(emg_data.iloc[:,3:])
    y = np.array(emg_data.Label2)
    #x,y = dp.generate_window_slide_data(emg_data,width=width,stride=stride,scaler=False,same_label=True)
    X += x.tolist()
    Y += y.tolist()
    i += 1
    print('i:%d'%i,x.shape)
end = time.time()
duration = end-start
print('Duration: %f'%(duration))

i:1 (16583, 8)
i:2 (4489, 8)
i:3 (35121, 8)
i:4 (31027, 8)
i:5 (30783, 8)
i:6 (159539, 8)
i:7 (225455, 8)
i:8 (362273, 8)
i:9 (28868, 8)
i:10 (20878, 8)
i:11 (25322, 8)
i:12 (24242, 8)
i:13 (21728, 8)
i:14 (27916, 8)
i:15 (27146, 8)
i:16 (25947, 8)
i:17 (26203, 8)
i:18 (89500, 8)
i:19 (70282, 8)
i:20 (79595, 8)
i:21 (73280, 8)
Duration: 4.539805


In [10]:
def train_model(model,feature,y,binary=True,file=None):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==3)|(y==4)|(y==6))
        ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y.copy()
        #ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}
    x_full,x_test,y_full,y_test = train_test_split(np.array(feature)[ind,:],y_01,
                                                   test_size=0.2,
                                                   random_state=123,
                                                   shuffle=True)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.25,
                                                       random_state=555,
                                                       shuffle=True)
    
    #sm = BorderlineSMOTE(random_state=50,kind='borderline-1')
    #sm = SMOTE(random_state=50)
    #print(y_full.shape)
    #x_full,y_full = sm.fit_resample(x_full,y_full)
    #print(y_full_n.shape)
    x_full = np.reshape(x_full,(-1,128*8))
    sc = StandardScaler(with_mean=False)
    #sc = MinMaxScaler((0,1))
    x_train = sc.fit_transform(x_full)
    x_train = x_train.reshape((-1,128,8))
    #pca = PCA(n_components=150)
    #x_train = pca.fit_transform(x_train)
    #x_valid = sc.transform(x_valid)
    x_test = np.reshape(x_test,(-1,128*8))
    x_test = sc.transform(x_test)
    x_test = x_test.reshape((-1,128,8))
    #x_test = pca.transform(x_test)
    #x_train = x_full
    
    early_stopping = callbacks.EarlyStopping(patience = 10,
                                             monitor = 'val_accuracy', 
                                             restore_best_weights=True)
    history = model.fit(x_train,y_full,validation_data=[x_test,y_test],
                        epochs=300,batch_size=500,class_weight=cw,
                        callbacks=[early_stopping],
                        shuffle=True)
    
    if binary:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #test = metrics.accuracy_score(y_test,y_pred_t>0.5)
        
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(y_valid,np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))
        #train = metrics.accuracy_score(y_full,y_pred_ta>0.5)
        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
        
        #print('train: \n',metrics.confusion_matrix(y_full,y_pred_ta>0.5))
        #print('test: \n',metrics.confusion_matrix(y_test,y_pred_t>0.5))

    else:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
    print('test:%f'%test)
    #print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,test,sc#,pca

In [ ]:
feature_re = feature_sc.reshape((-1,X.shape[1],1))
x_full,x_test,y_full,y_test = train_test_split(np.array(feature_re),np.array(y_01),test_size=0.2,random_state=123)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555)

In [11]:
input_ = layers.Input(shape=[128,8])
lstm1 = layers.LSTM(50,return_sequences=True)(input_)
drop1 = layers.Dropout(0.2)(lstm1)
lstm2 = layers.LSTM(50,return_sequences=True)(drop1)
drop2 = layers.Dropout(0.2)(lstm2)
#lstm3 = layers.LSTM(50,return_sequences=True)(drop2)
#drop3 = layers.Dropout(0.2)(lstm3)
lstm4 = layers.LSTM(50)(drop2)
drop4 = layers.Dropout(0.2)(lstm4)
dense1 = layers.Dense(64,activation='relu')(drop4)
drop5 = layers.Dropout(0.2)(dense1)
dense2 = layers.Dense(32,activation='relu')(drop5)
output = layers.Dense(3,activation='softmax')(dense2)
model = Model(inputs=[input_],outputs=[output])

In [12]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=100,batch_size=500,
                   callbacks=[early_stopping])

In [13]:
train,test,sc = train_model(model,np.array(x),np.array(y),False)

Train on 8872 samples, validate on 2219 samples
Epoch 1/300
8872/8872 [==============================] - 88s 10ms/sample - loss: 0.9841 - accuracy: 0.4876 - val_loss: 0.8857 - val_accuracy: 0.5002
Epoch 2/300
8872/8872 [==============================] - 82s 9ms/sample - loss: 0.8572 - accuracy: 0.5571 - val_loss: 0.8288 - val_accuracy: 0.6471
Epoch 3/300
8872/8872 [==============================] - 81s 9ms/sample - loss: 0.8051 - accuracy: 0.6443 - val_loss: 0.7573 - val_accuracy: 0.6670
Epoch 4/300
8872/8872 [==============================] - 81s 9ms/sample - loss: 0.7433 - accuracy: 0.6677 - val_loss: 0.7142 - val_accuracy: 0.6791
Epoch 5/300
8872/8872 [==============================] - 81s 9ms/sample - loss: 0.7212 - accuracy: 0.6787 - val_loss: 0.6996 - val_accuracy: 0.6886
Epoch 6/300
8872/8872 [==============================] - 87s 10ms/sample - loss: 0.7071 - accuracy: 0.6878 - val_loss: 0.6952 - val_accuracy: 0.7008
Epoch 7/300
8872/8872 [==============================] - 81s 9

KeyboardInterrupt: 

In [ ]:
feature2_re = feature2_sc.reshape((-1,feature.shape[1],1))

In [ ]:
test_model(model,x_test,y_test)

In [113]:
cost_matrix=np.array([[0,1,1,1],
                  [1,0,1,1],
                  [10,100,0,10],
                  [1.,1.,1,0]])
cost_matrix=np.array([[0,2.],
                  [5.,0]])

In [58]:
P0 = 55080/(55080+8181)
P1 = 8181/(55080+8181)
P = [P0,P1]
cost_vec = np.zeros(2)
for i in range(2):
    for j in range(2):
        if i == j:
            continue
        cost_vec[i]+=1/(1-P[i])*P[j]*cost_matrix[i,j]

In [59]:
cost_vec

array([2., 5.])

In [112]:
tf.transpose(t)

<tf.Tensor: id=1567100, shape=(4, 4), dtype=float64, numpy=
array([[  0.,   1.,  10.,   1.],
       [  1.,   0., 100.,   1.],
       [  1.,   1.,   0.,   1.],
       [  1.,   1.,  10.,   0.]])>

In [116]:
np.reshape(cost_matrix,(-1,1))

array([[0.],
       [2.],
       [5.],
       [0.]])